In [ ]:
import dautil as dl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import HTML

In [ ]:
context = dl.nb.Context('exp_smoothing')
lr = dl.nb.LatexRenderer(chapter=6, start=3, context=context)
lr.render(r's_{t}= \alpha x_{t} + (1-\alpha)s_{t-1},\ t>0')
lr.render(r's_{t} = \alpha x_{t} + (1-\alpha)(s_{t-1} + b_{t-1}')
lr.render(r'b_{t} = \beta (s_t - s_{t-1}) + (1-\beta)b_{t-1}')

In [ ]:
def grid_mse(i, j, devs):
    alpha = 0.1 * i
    beta = 0.1 * j
    cell = dl.ts.double_exp_smoothing(devs.values, alpha, beta)

    return dl.stats.mse(devs, cell)

In [ ]:
wind = dl.data.Weather.load()['WIND_SPEED'].dropna()
wind = dl.ts.groupby_year(wind).mean()
devs = dl.ts.rolling_deviations(wind, 12).dropna()

In [ ]:
%matplotlib inline
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 2, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
sp.label(ylabel_params=dl.data.Weather.get_header('WIND_SPEED'))
sp.ax.plot(wind.index, wind)

cp = dl.plotting.CyclePlotter(sp.next_ax())
cp.plot(devs.index, devs, label='Rolling Deviations')
cp.plot(devs.index, dl.ts.exp_smoothing(devs.values, 0.7), label='Smoothing')
sp.label()

alphas = 0.01 * np.arange(1, 100)
errors = [dl.stats.mse(devs, dl.ts.exp_smoothing(devs.values, alpha)) 
          for alpha in alphas]
sp.label(advance=True)
sp.ax.plot(alphas, errors)

sp.label(advance=True)
rng = range(1, 10)
df = dl.report.map_grid(rng, rng, ["alpha", "beta", "mse"], grid_mse, devs) 
sns.heatmap(df, cmap='Blues', square=True, annot=True, fmt='.1f',
            ax=sp.ax)

HTML(sp.exit())